In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from generate import generate
from transformers import AutoTokenizer, AutoModel


device = 'cuda'
model = AutoModel.from_pretrained('GSAI-ML/LLaDA-8B-Instruct', trust_remote_code=True, torch_dtype=torch.bfloat16).to(device).eval()
tokenizer = AutoTokenizer.from_pretrained('GSAI-ML/LLaDA-8B-Instruct', trust_remote_code=True)

/home/sagoyal/research/mdlm/LLaDA/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sagoyal/research/mdlm/LLaDA/.venv/lib64/python3.12/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 6/6 [00:29<00:00,  4.92s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
gen_length = 64
steps = 32
print('*' * 66)
print(f'**  Answer Length: {gen_length}  |  Sampling Steps: {steps}  **')
print('*' * 66)

conversation_num = 0
user_input = "Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?"

m = [{"role": "user", "content": user_input}]
user_input = tokenizer.apply_chat_template(m, add_generation_prompt=True, tokenize=False)
input_ids = tokenizer(user_input)['input_ids']
input_ids = torch.tensor(input_ids).to(device).unsqueeze(0)

if conversation_num == 0:
    prompt = input_ids
else:
    prompt = torch.cat([prompt, input_ids[:, 1:]], dim=1)

******************************************************************
**  Answer Length: 64  |  Sampling Steps: 32  **
******************************************************************


In [20]:
from generate import generate

out = generate(model, 
    prompt, 
    steps=steps, 
    gen_length=gen_length, 
    block_length=gen_length, 
    temperature=0.0, 
    cfg_scale=0., 
    remasking='low_confidence')

# print("Direct out: ", out)

answer = tokenizer.batch_decode(out[:, prompt.shape[1]:], skip_special_tokens=True)[0]
print(f"Bot's reply: {answer}")

# remove the <EOS>
prompt_out = out[out != 126081].unsqueeze(0)
conversation_num += 1
print('-----------------------------------------------------------------------')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tensor([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2]], device='cuda:0')

Top-10 tokens per position at step 0:

Position 133:
  Rank   Token                          Probability 
  ------------------------------------------------
  1                                     0.259766     ◄ SELECTED
  2       is                            0.202148     
  3      :                              0.189453     
  4      <|endoftext|>                  0.147461     
  5      5                              0.017578     
  6       that                          0.014587     
  7      ####                           0.014587     
  8      0                              0.012146     
  9      <|eot_id|>                     0.011353     
  10      in                            0.008301     

Position 134:
  Rank   Token                          Probability 
  ------------------------------------------------
  1                                     